In [1]:
import pandas as pd
from anytree import Node, RenderTree
import random
import string

from functions import *
from Twitter_functions import *

In [2]:
with open('Authentication/twitter_bearer_token.txt', 'r', encoding="utf8") as f:
    token = f.read()

header = create_Twitter_headers(token)
max_results = 100

# The Straits Times
## Get Tweets

In [3]:
ST_id = '37874853'
ST_tweets = getTweets(ST_id, header)

generateTweetInformation(ST_tweets, header)

Tweet ID: 1514143232891924481 Time: 2022-04-13T07:27:32.000Z
Jail for man who misappropriated over $71k while working for two firms https://t.co/ASiF40bHda 

Tweet ID: 1514140594439786496 Time: 2022-04-13T07:17:03.000Z
RT @ST_LifeTweets: New film, new Grindelwald: Mads Mikkelsen enters the Wizarding World https://t.co/1YPKZCgLWI 

Tweet ID: 1514138707472748547 Time: 2022-04-13T07:09:33.000Z
RT @STForeignDesk: Boeing 737 Max simulator glitches prompt India to ground pilots https://t.co/BQKh6CdZui 

Tweet ID: 1514138702435393536 Time: 2022-04-13T07:09:32.000Z
RT @STForeignDesk: Hong Kong rules out any plan to roll back home isolation https://t.co/hSbFAJbo5d 

Tweet ID: 1514136820681826306 Time: 2022-04-13T07:02:04.000Z
RT @stbusinessdesk: Celebrities toss $119 million to crypto payments company MoonPay https://t.co/ifsh9QwAvf 

Tweet ID: 1514132165981716484 Time: 2022-04-13T06:43:34.000Z
RT @ST_LifeTweets: Chris Pine, Thandiwe Newton spar in spy thriller All The Old Knives https://t.co/m

In [4]:
# query = '@straits_times'
# tweets_url = f'https://api.twitter.com/2/tweets/search/recent?query={query}&max_results={max_results}'
# connect_to_endpoint(tweets_url, header)

## Get Conversation

In [5]:
conversation_id = '1513829591038627840'
ST_conversation_data = getConversation(conversation_id, max_results, header)

In [6]:
ST_result = getTweetComments(ST_conversation_data)

User ID: 185838029 Time: 2022-04-13T05:32:26.000Z
In reply to: 37874853
@straits_times The Emperor has no clothes. 

User ID: 1234346092507975680 Time: 2022-04-13T05:21:54.000Z
In reply to: 37874853
@straits_times https://t.co/ufQlZRAsdW 

User ID: 294650098 Time: 2022-04-13T04:22:16.000Z
In reply to: 37874853
@straits_times Just my opinion which is I trusted more during Mr Lee Kuan Yew's reign. The world now is in a complete mess and we are heading towards I dont Know! 

User ID: 1264999890049921024 Time: 2022-04-13T04:10:21.000Z
In reply to: 37874853
@straits_times https://t.co/7qOGJhREnn 

User ID: 1432612826758017025 Time: 2022-04-13T00:34:49.000Z
In reply to: 37874853
@straits_times Zero trust in the PAP. Whole bunch of liars. 

User ID: 2668705165 Time: 2022-04-12T16:50:50.000Z
In reply to: 37874853
@straits_times fool's paradise 

User ID: 1349288520750309376 Time: 2022-04-12T16:46:31.000Z
In reply to: 37874853
@straits_times Force vaccinations. While saying that we trust the go

In [7]:
ST_df = processTwitterDataframe(ST_result, ST_id, conversation_id, uri, False)
ST_df.head()

Conversation has been updated in the database.


,id,timestamp,reply_to,comment,social_media,conversation_id,head_id,url,link_title
0,10000,2022-04-13T05:32:26.000Z,37874853,@straits_times The Emperor has no clothes.,Twitter,1513829591038627840,37874853,,
1,10001,2022-04-13T05:21:54.000Z,37874853,@straits_times https://t.co/ufQlZRAsdW,Twitter,1513829591038627840,37874853,https://twitter.com/HuiYan01565633/status/1514...,[twitter.com]
2,10002,2022-04-13T04:22:16.000Z,37874853,@straits_times Just my opinion which is I trus...,Twitter,1513829591038627840,37874853,,
3,10003,2022-04-13T04:10:21.000Z,37874853,@straits_times https://t.co/7qOGJhREnn,Twitter,1513829591038627840,37874853,https://twitter.com/KuroBoyX0/status/151409360...,[twitter.com]
4,10004,2022-04-13T00:34:49.000Z,37874853,@straits_times Zero trust in the PAP. Whole bu...,Twitter,1513829591038627840,37874853,,


# Channel NewsAsia / Mothership
## Get Tweets

In [8]:
# CNA_tweets = getTweets('38400130', header) # CNA
CNA_tweets = getTweets('1619325942', header) # Mothership

generateTweetInformation(CNA_tweets, header)

Tweet ID: 1514142353753206785 Time: 2022-04-13T07:24:03.000Z
Some SCDF frontliners &amp; enforcement officers to wear body cameras
https://t.co/pP6RdKDxmz https://t.co/yugXVIUkUB 

Tweet ID: 1514127531384877060 Time: 2022-04-13T06:25:09.000Z
S'pore-JB Causeway jam likely this Good Friday long weekend, 3-hour clearance possible https://t.co/Dk1A9zoJJ0 https://t.co/eSTaq19JWy 

Tweet ID: 1514127404595224577 Time: 2022-04-13T06:24:39.000Z
S'poreans, long-term pass holders coming from Johor need not fill SG Arrival Card https://t.co/vdlsoVNwLZ https://t.co/Pz5mmS3b9D 

Tweet ID: 1514125050311442435 Time: 2022-04-13T06:15:17.000Z
nice https://t.co/15lZdKI749 

Tweet ID: 1514120130241843202 Time: 2022-04-13T05:55:44.000Z
2 Ramadan bazaars to start in Woodlands on Apr. 15 &amp; 16 https://t.co/9GofWJKoOd https://t.co/HXssdLB1Kh 

Tweet ID: 1514117974822572032 Time: 2022-04-13T05:47:10.000Z
Land border reopens: M'sians in S'pore hope to restart daily commute for lower costs &amp; time with fam

## Get Conversation

In [9]:
conversation_id = '1513446924401410050'
CNA_conversation_data = getConversation(conversation_id, max_results, header)

result = getTweetComments(CNA_conversation_data)

User ID: 1480079619940175872 Time: 2022-04-12T09:22:53.000Z
In reply to: 1619325942
@MothershipSG Anil. Raj 

User ID: 1427808774354464770 Time: 2022-04-12T08:14:21.000Z
In reply to: 1411490772482883587
@sammie_artz @aguyandhisgirl1 @MothershipSG The dangers of reckless driving 

User ID: 1411490772482883587 Time: 2022-04-12T07:22:42.000Z
In reply to: 1427808774354464770
@Cat_I19 @aguyandhisgirl1 @MothershipSG Awareness of what? Grief? Sadness? I don't get it. 

User ID: 933503220 Time: 2022-04-12T05:45:53.000Z
In reply to: 1619325942
@MothershipSG Rip 

User ID: 1238746781615132673 Time: 2022-04-12T04:57:47.000Z
In reply to: 949722554247274496
@Taetaemonie @MothershipSG 😭😭😭😭😭那是我好女媳 就像儿子一样 

User ID: 1238746781615132673 Time: 2022-04-12T04:55:50.000Z
In reply to: 1513417869354168331
@D_rkn1ght @MothershipSG 😭😭😭😭😭😭😭😭😭forever 

User ID: 1238746781615132673 Time: 2022-04-12T04:54:52.000Z
In reply to: 1447168812290244610
@Zizie64 @MothershipSG 😭😭😭😭😭my son in law 

User ID: 1234746671365378

In [10]:
CNA_df = processTwitterDataframe(result, '1619325942', conversation_id, uri, False)
CNA_df.head()

Conversation has been updated in the database.


,id,timestamp,reply_to,comment,social_media,conversation_id,head_id,url,link_title
0,10000,2022-04-12T09:22:53.000Z,1619325942,@MothershipSG Anil. Raj,Twitter,1513446924401410050,1619325942,,
1,10001,2022-04-12T08:14:21.000Z,10002,@sammie_artz @aguyandhisgirl1 @MothershipSG Th...,Twitter,1513446924401410050,1619325942,,
2,10002,2022-04-12T07:22:42.000Z,10001,@Cat_I19 @aguyandhisgirl1 @MothershipSG Awaren...,Twitter,1513446924401410050,1619325942,,
3,10003,2022-04-12T05:45:53.000Z,1619325942,@MothershipSG Rip,Twitter,1513446924401410050,1619325942,,
4,10004,2022-04-12T04:57:47.000Z,10035,@Taetaemonie @MothershipSG 😭😭😭😭😭那是我好女媳 就像儿子一样,Twitter,1513446924401410050,1619325942,,
